In [40]:
from math import radians, cos, sin, asin, sqrt, pow

def great_circle_distance2(lat1, lon1, lat2, lon2):
    """
    Calculate distance on plane
    """
    dx = lat1-lat2
    dy = lon1-lon2
    d = sqrt(dx*dx+dy*dy)
    if d < 0:
        raise Exception("Negative distance!")
    return d

def great_circle_distance1(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# Mean Square Error
# locations: [ (lat1, long1), ... ]
# distances: [ distance1, ... ]
def mse(x, locations, distances):
    mse = 0.0
    for location, distance in zip(locations, distances):
        distance_calculated = great_circle_distance2(x[0], x[1], location[0], location[1])
        mse += pow(distance_calculated - distance, 2.0)
    return mse / len(distances)

#locations = [(0,0),(1,0),(0,1)]
#distances = [1.414,1,1]
#x = (1.414,0)
#error1 = mse(x, locations, distances)
#print(error1)

In [72]:
beacon1 = {
    'distance': 0.7,
    'location': (0,0)
}
beacon2 = {
    'distance': 0.7,
    'location': (1,0)
}
beacon3 = {
    'distance': 0.7,
    'location': (0,1)
}
data = [beacon1,beacon2,beacon3]
#print(data[0]['location'])

# Initial point: the point with the closest distance
min_distance     = float('inf')
closest_location = None
for member in data:
    # A new closest point!
    if member['distance'] < min_distance:
        min_distance = member['distance']
        closest_location = member['location']
initial_location = closest_location
print("initial_location: ",initial_location)

# initial_location: (lat, long)
# locations: [ (lat1, long1), ... ]
# distances: [ distance1,     ... ] 
locations = [(0,0),(1,0),(0,1)]
distances = [0.7,0.7,0.7]
from scipy.optimize import minimize, rosen, rosen_der
result = minimize(
    mse,                         # The error function
    initial_location,            # The initial guess
    args=(locations, distances), # Additional parameters for mse
    method='L-BFGS-B',           # The optimisation algorithm
    options={
        'ftol':1e-6,         # Tolerance
        'maxiter': 1e+7      # Maximum iterations
    })
location = result.x
print("location: ",location)
print(result)

initial_location:  (0, 0)
location:  [ 0.4950742   0.49507419]
      fun: 3.40034108974248e-05
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -1.61837503e-08,  -2.33150901e-08])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 18
      nit: 5
   status: 0
  success: True
        x: array([ 0.4950742 ,  0.49507419])


In [50]:
import openpyxl

# Data read from table "dw_tms_interstellar_ins_beacon_data_hour"
class entry():
    def __init__(self):
        self.id = None
        self.user_id = None
        self.rssi = None
        self.major = None
        self.minor = None
        self.date = None
        self.svrHour = None
        self.svrMnut = None
        self.svrScnd = None
        self.crtShop = None
        self.lclHour = None
        self.lclMnut = None
        self.lclScnd = None
        self.latitude = None
        self.longitude = None

        
# Calculate the time delta in seconds
def timeDelta(lateHour, lateMnut, lateScnd, erlyHour, erlyMnut, erlyScnd):
    return (lateHour-erlyHour)*3600 + (lateMnut-erlyMnut)*60 + lateScnd - erlyScnd
        
    
def readBeaconFile(fileName):
    
    # Beacon RSSI at fixed distance
    wb = openpyxl.load_workbook(fileName+".xlsx")
    sheet = wb.get_sheet_by_name('Sheet0')
    
    row_count = sheet.max_row
    # column_count = sheet.max_column
    
    dataNum = row_count - 1
    
    beacon_data = [entry() for i in range(dataNum)]
    maxHour = 0
    maxMnut = 0
    maxScnd = 0
    minHour = 23
    minMnut = 59
    minScnd = 59
    
    
    for i in range(0,dataNum):
        beacon_data[i].id = sheet.cell(row=i+2,column=1).value
        beacon_data[i].user_id = sheet.cell(row=i+2,column=2).value
        beacon_data[i].rssi = sheet.cell(row=i+2,column=9).value
        beacon_data[i].latitude = sheet.cell(row=i+2,column=11).value
        beacon_data[i].longitude = sheet.cell(row=i+2,column=12).value
        beacon_data[i].uuid = sheet.cell(row=i+2,column=13).value
        beacon_data[i].major = int(sheet.cell(row=i+2,column=14).value)
        beacon_data[i].minor = int(sheet.cell(row=i+2,column=15).value)
        dateStr = sheet.cell(row=i+2,column=25).value
        if dateStr is None:
            print("ERROR: NoneType found!!!, i:", i)
            return
        d1 = dateStr.split(" ")                            
        beacon_data[i].date = d1[0]
        time1 = d1[1].split(":")
        time2 = time1[2].split(".")
        beacon_data[i].lclHour = int(time1[0])
        beacon_data[i].lclMnut = int(time1[1])
        beacon_data[i].lclScnd = int(time2[0])
        
        if timeDelta(beacon_data[i].lclHour, beacon_data[i].lclMnut, 
                     beacon_data[i].lclScnd, maxHour, maxMnut, maxScnd) > 0:
            maxHour = beacon_data[i].lclHour
            maxMnut = beacon_data[i].lclMnut
            maxScnd = beacon_data[i].lclScnd
        if timeDelta(beacon_data[i].lclHour, beacon_data[i].lclMnut, 
                     beacon_data[i].lclScnd, minHour, minMnut, minScnd) < 0:
            minHour = beacon_data[i].lclHour
            minMnut = beacon_data[i].lclMnut
            minScnd = beacon_data[i].lclScnd                           
        beacon_data[i].crtShop = int(sheet.cell(row=i+2,column=24).value)
    
    strtTime = [minHour,minMnut,minScnd]
    endTime = [maxHour,maxMnut,maxScnd]
    
    return beacon_data, strtTime, endTime

In [71]:
# Here we estimate the parameters in the rssi2distance function
# Formula in [7]:
#    rssi = -65 - 10n*log(d)

import numpy as np
# Read file
fileName = "myTest0714"
beacon_data, strtTime, endTime = readBeaconFile(fileName)
rssi = [[],[],[],[],[],[],[]]

for data in beacon_data:
    if data.crtShop == 0:
        continue
    rssi[data.crtShop-1].append(data.rssi)

dist = [[],[],[],[],[],[],[]]
for i in range(0,7):
    num = len(rssi[i])
    delta = 12/(num-1)
    if i < 4:
        for j in range(0,num):
            dist[i].append(j*delta+1)
    else:
        for j in range(0,num):
            dist[i].append(13-j*delta)
    
n = []

for i in range(0,7):
    x = -10*np.log10(dist[i])
    add65 = [65 for i in range(len(x))]
    y = [a + b for a, b in zip(rssi[i], add65)]
    n_tilde = np.polyfit(x, y, 1)
    n.append(n_tilde)
     
print(rssi)
print(dist)
print(n)

# n is around 2.8

[[-66.0, -66.0, -60.0, -60.0, -84.0, -71.0, -68.0, -64.0, -75.0, -75.0, -89.0, -78.0, -91.0, -92.0, -95.0, -86.0, -86.0, -86.0, -91.0, -91.0], [-56.0, -55.0, -65.0, -77.0, -78.0, -69.0, -82.0, -82.0, -82.0, -93.0, -90.0, -91.0, -91.0, -87.0, -88.0, -82.0], [-61.0, -70.0, -70.0, -67.0, -66.0, -69.0, -67.0, -66.0, -77.0, -87.0, -87.0, -83.0, -86.0, -90.0, -88.0, -88.0, -83.0, -86.0, -88.0, -88.0, -91.0], [-54.0, -56.0, -58.0, -67.0, -68.0, -70.0, -70.0, -70.0, -83.0, -77.0, -84.0, -87.0, -80.0, -92.0, -81.0, -81.0, -82.0, -82.0, -77.0, -76.0, -86.0, -85.0], [-84.0, -87.0, -91.0, -86.0, -81.0, -76.0, -82.0, -77.0, -75.0, -75.0, -77.0, -76.0, -76.0, -65.0, -61.0, -61.0, -73.0], [-92.0, -86.0, -83.0, -83.0, -85.0, -85.0, -78.0, -77.0, -76.0, -58.0], [-85.0, -79.0, -79.0, -83.0, -77.0, -77.0, -85.0, -80.0, -77.0, -77.0, -71.0, -70.0, -68.0, -79.0, -75.0, -70.0, -59.0]]
[[1.0, 1.631578947368421, 2.263157894736842, 2.894736842105263, 3.526315789473684, 4.157894736842105, 4.789473684210526, 5.4

In [65]:
import numpy as np
x = [0,1,2]
y = [1.0001,5.9999,15.0]
x = np.array(x)
y = np.array(y)

z = np.polyfit(x, y, 2)
print(z)

print(np.log10(1))

[ 2.00015  2.99965  1.0001 ]
0.0
